# Nonlinear time series analysis of palaeoclimate proxy records

_by Norbert Marwan, Jonathan F. Donges, Deniz Eroglu, published in Quaternary Science Reviews, 2021
(script implemented by Deniz Eroglu, KH Univ Istanbul)_

Script for nonlinear time series analysis of the palaeoclimate proxy records used in the paper. It calculates the number of potential wells $n_u$, the entropy of the data $S$, the order pattern (permutation) entropy $S_{order}$, the recurrence quantification/network measures $DET$, $LAM$, $\mathcal{T}$, and $\mathcal{L}$ as well as the visibility graph based irreversibility test statsitics $p(k)$ and $p(\mathcal{C})$.

The data file can be simply exchanged by own data file.

## Import necessary packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import nonlinear_timeseries_analysis as ntsa # Time series analysis code [nonlinear_timeseries_analysis.py], should be in the same folder with this jupyter notebook.
from pyunicorn.timeseries import RecurrenceNetwork, VisibilityGraph  # pyunicorn - http://www.pik-potsdam.de/~donges/pyunicorn/
import scipy.stats
%matplotlib inline

## Loading data file

Data file can be replaced by any other data file (in ASCII text format). Ensure to assign the column of the time points and the column of the proxy values to the variable `x` first and second column.

In [4]:
name = 'Data/ODP659_dust' # data file in text format,
y = np.loadtxt(name + '.txt')
n = len(y) # length of time series
x = np.zeros([n,2])
x[:,0] = deepcopy(y[:,8]) # time points
x[:,1] = deepcopy(y[:,10]) # flux data
del y

## Set parameters and prepare results variable

We prepare the variables that will store the results of the different time series analysis methods. Some of the methods and the windowing procedure require selection of parameters.

In [5]:
# Arrange the sliding windows
min_t = min(x[:,0])
max_t = max(x[:,0])

win_size = 410 # window size for sliding windows 
overlap = win_size/10 # overlap of sliding windows
win = np.arange(min_t ,max_t-win_size,overlap)
N = len(win) # number of sliding windows

# Average window length
average_sampling_time = np.diff(x[:,0]).mean()
average_win_points = int(win_size/average_sampling_time)

# Storage matrix for the measures (results)
NofMeasures = 10
results = np.zeros([N,NofMeasures])

# Constants for the measures
# Potential Analysis
sigma = 1.5

# Shannon Entropy
bins_shannon = 20

# Block Entropy
#block_bins = 9

# Recurrence Plot
eps = 0.05
d = 3   # dimension of embedding
t = 2   # time delay

# Visibility Graph
#  Toggle calculation of horizontal visibility graphs (slower alternative are standard visibility graphs)
HORIZONTAL = True

# Keep histogram for significance
P_l = np.zeros(2*average_win_points)
P_v = np.zeros(2*average_win_points)
P_block = np.array([])
P_order = np.array([])

## Run analysis

The time series analysis is performed in a running window. The results of the analyses are stored in the matrix `results`, with dimension 1 corresponding to the time of the window and dimension 2 corresponding to the specific method/measure:

|Position|Measure|
|--------|-------|
|0       |time point of window $t$ |
|1       |number of potential wells $n_u$|
|2       |value entropy $S$|
|3       |order entropy $S_{order}$|
|4       |transitivity $\mathcal{T}$|
|5       |determinism $DET$|
|6       |laminarity $LAM$|
|7       |average path length $\mathcal{L}$|
|8       |irrev. statistics $p(k)$|
|9       |irrev. statistics $p(\mathcal{C})$|


In [6]:
m = 0 # counter
for i in win:

    # Aranging the windowed time series
    y = ntsa.indices(x[:,0], lambda y: y>i and y<i+win_size)

    # Recurrence Based Analysis
    net = RecurrenceNetwork(x[y,1],recurrence_rate=eps, silence_level=3, dim = d, tau=t)
    P_l_temp = net.diagline_dist()
    P_v_temp = net.vertline_dist()
    for j in range(len(P_l_temp)):
        P_l[j] += P_l_temp[j]
        P_v[j] += P_v_temp[j]
    P_order_temp = ntsa.symbolicPerm(x[y,1], d,t)
    P_order =  sorted(np.append(P_order, P_order_temp))

    # Visibility Graph
    vg = VisibilityGraph(time_series=x[y,1], timings=x[y,0], horizontal=HORIZONTAL, silence_level=3)
    l_k_obs = vg.retarded_degree()
    r_k_obs = vg.advanced_degree()    
    l_C_obs = vg.retarded_local_clustering()
    r_C_obs = vg.advanced_local_clustering()

    # store results
    results[m,0] = (max(x[y,0]) + min(x[y,0]))/2
    results[m,1] = ntsa.potential(x[y,1], sigma)
    results[m,2] = ntsa.shannonEntropy(x[y,1], bins_shannon)
    results[m,3] = ntsa.symbolicEntropy(P_order_temp)
    results[m,4] = net.transitivity()
    results[m,5] = net.determinism()
    results[m,6] = net.laminarity()
    results[m,7] = net.average_path_length()    
    results[m,8] = scipy.stats.ks_2samp(l_k_obs, r_k_obs)[1]
    results[m,9] = scipy.stats.ks_2samp(l_C_obs, r_C_obs)[1]
    m += 1

## Significance for RQA and Entropy cases

Confidence intervals (variables `left` and `right`) are estimated using specific surrogate tests and from the 5% and 95% percentiles of the empirical test distributions.

In [15]:
significance_measures = 3
realisations = 1000
surrogate_results = np.zeros([realisations,significance_measures])
left = np.zeros([NofMeasures])
right = np.zeros([NofMeasures])

m = 0

n_l = int(sum(P_l)/win.size)                # Number of recurrence plot components (diagonal lines)
n_v = int(sum(P_v)/win.size)                # Number of recurrence plot components (diagonal lines)
n_order =  int(len(P_order)/win.size)

for m in range(realisations):
    y = [np.random.randint(0,n) for r in range(average_win_points)]
    net = RecurrenceNetwork(x[y,1],recurrence_rate=eps, silence_level=3, dim = d, tau=t)
    surrogate_results[m,0] = ntsa.shannonEntropy(x[y,1], bins_shannon)
    surrogate_results[m,1] = net.transitivity()
    surrogate_results[m,2] = net.average_path_length()

left[2], right[2] = ntsa.significance(surrogate_results[:,0])
left[3], right[3] =  ntsa.block_significance(P_order, n_order)
left[4], right[4] = ntsa.significance(surrogate_results[:,1])
left[5], right[5] = ntsa.rp_significance(P_l, n_l,measure='determinism')
left[6], right[6] = ntsa.rp_significance(P_v, n_v,measure='laminarity')
left[7], right[7] = ntsa.significance(surrogate_results[:,2])



## Plotting the results

In [16]:
plt.figure(num=None, figsize=(15, 15), dpi=100, facecolor='w', edgecolor='k')
plt.subplot2grid((NofMeasures,1), (0,0))
plt.plot(x[:,0],x[:,1])
plt.ylabel(name)
plt.title('window len =' + str(win_size) + ' yrs BP,  overlap=' + str(overlap)+' yrs BP, embedding dim =' + str(d) + ', time delay =' + str(t) )
plt.xlim(min_t,max_t)


plt.subplot2grid((NofMeasures,1), (1,0))
plt.plot(results[:,0],results[:,1])
plt.xlim(min_t,max_t)
plt.ylabel(r'$\mathcal{n}_u$',fontsize=14)


plt.subplot2grid((NofMeasures,1), (2,0))
plt.plot(results[:,0],results[:,2])#/np.max(results[:,2]),'o-')
plt.axhspan(left[2], right[2], facecolor='g', alpha=0.5)
plt.fill_between(results[:,0], left[2], results[:,2], where=results[:,2]<=left[2], facecolor='green', interpolate=True)
plt.fill_between(results[:,0], right[2], results[:,2], where=results[:,2]>=right[2], facecolor='green', interpolate=True)
plt.xlim(min_t,max_t)
plt.ylabel(r'$S$',fontsize=14)
    
    
plt.subplot2grid((NofMeasures,1), (3,0))
plt.plot(results[:,0],results[:,3])#/np.max(results[:,4]),'o-')
plt.axhspan(left[3], right[3], facecolor='g', alpha=0.5)
plt.fill_between(results[:,0], left[3], results[:,3], where=results[:,3]<=left[3], facecolor='green', interpolate=True)
plt.fill_between(results[:,0], right[3], results[:,3], where=results[:,3]>=right[3], facecolor='green', interpolate=True)
plt.xlim(min_t,max_t)
plt.ylabel(r'$S_{Order}$',fontsize=14) 


plt.subplot2grid((NofMeasures,1), (4,0))
plt.plot(results[:,0],results[:,4])
plt.axhspan(left[4], right[4], facecolor='g', alpha=0.5)
plt.fill_between(results[:,0], left[4], results[:,4], where=results[:,4]<=left[4], facecolor='green', interpolate=True)
plt.fill_between(results[:,0], right[4], results[:,4], where=results[:,4]>=right[4], facecolor='green', interpolate=True)
plt.xlim(min_t,max_t)
plt.ylabel(r'$\mathcal{T}$',fontsize=14)


plt.subplot2grid((NofMeasures,1), (5,0))
plt.plot(results[:,0],results[:,5])
plt.axhspan(left[5], right[5], facecolor='g', alpha=0.5)
plt.fill_between(results[:,0], left[5], results[:,5], where=results[:,5]<=left[5], facecolor='green', interpolate=True)
plt.fill_between(results[:,0], right[5], results[:,5], where=results[:,5]>=right[5], facecolor='green', interpolate=True)
plt.xlim(min_t,max_t)
plt.ylabel(r'$DET$',fontsize=14)


plt.subplot2grid((NofMeasures,1), (6,0))
plt.plot(results[:,0],results[:,6])
plt.axhspan(left[6], right[6], facecolor='g', alpha=0.5)
plt.fill_between(results[:,0], left[6], results[:,6], where=results[:,6]<=left[6], facecolor='green', interpolate=True)
plt.fill_between(results[:,0], right[6], results[:,6], where=results[:,6]>=right[6], facecolor='green', interpolate=True)
plt.xlim(min_t,max_t)
plt.ylabel(r'$LAM$',fontsize=14)


plt.subplot2grid((NofMeasures,1), (7,0))
plt.plot(results[:,0],results[:,7])
plt.axhspan(left[7], right[7], facecolor='g', alpha=0.5)
plt.fill_between(results[:,0], left[7], results[:,7], where=results[:,7]<=left[7], facecolor='green', interpolate=True)
plt.fill_between(results[:,0], right[7], results[:,7], where=results[:,7]>=right[7], facecolor='green', interpolate=True)
plt.xlim(min_t,max_t)
plt.ylabel(r'$\mathcal{L}$',fontsize=14)


plt.subplot2grid((NofMeasures,1), (8,0))
plt.plot(results[:,0],results[:,8])
plt.xlim(min_t,max_t)
plt.ylabel(r'$p(k)$',fontsize=14)


plt.subplot2grid((NofMeasures,1), (9,0))
plt.plot(results[:,0],results[:,9])
plt.xlim(min_t,max_t)
plt.ylabel(r'$p(C)$',fontsize=14)


plt.xlabel(r'Time(yrs BP)',fontsize=14);